# MNIST with CNN using Keras

## Setup

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

import boto3
import time

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./.data/', one_hot=True)

Extracting ./.data/train-images-idx3-ubyte.gz
Extracting ./.data/train-labels-idx1-ubyte.gz
Extracting ./.data/t10k-images-idx3-ubyte.gz
Extracting ./.data/t10k-labels-idx1-ubyte.gz


In [2]:
X_train = mnist.train.images
X_test = mnist.test.images
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')/255

y_train = mnist.train.labels
y_test = mnist.test.labels

## Define network

In [3]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, activation='relu', input_shape=(1,28,28), dim_ordering='th'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Train

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [5]:
t0 = time.time()
model.fit(X_train, y_train,
          batch_size=50,
          nb_epoch=10
         )
print "training time = {}".format(time.time() - t0)

Epoch 1/10
   50/55000 [..............................] - ETA: 90s - loss: 2.3026 - acc: 0.1000

/usr/local/lib/python2.7/dist-packages/mxnet-0.11.0-py2.7.egg/mxnet/module/bucketing_module.py:385: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.02). Is this intended?
  force_init=force_init)


55000/55000 [==============================] - 196s - loss: 1.3475 - acc: 0.5095   
Epoch 2/10
 7550/55000 [===>..........................] - ETA: 172s - loss: 0.2919 - acc: 0.9102

KeyboardInterrupt: 

## Evaluate

In [6]:
score = model.evaluate(X_test, y_test, verbose=1)
print score

10000/10000 [==============================] - 10s    


## Save Model

In [11]:
model.save('./keras.hdf5')

s3 = boto3.client('s3')
s3.upload_file(
    './keras.hdf5',
    'jakechenawstemp',
    'keras.hdf5'
)